In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
torch.manual_seed(42)

In [4]:
df = pd.read_csv('D:/github/pytorch_/fashion-mnist_train.csv/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
device = torch.device('cuda')
print(f'Using device: {device}')

Using device: cuda


In [6]:
# train test split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# scaling the feautures
X_train = X_train/255.0
X_test = X_test/255.0

In [9]:
# create CustomDataset Class
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):

    return len(self.features)

  def __getitem__(self, index):

    return self.features[index], self.labels[index]

In [10]:
# create train_dataset object
train_dataset = CustomDataset(X_train, y_train)

In [11]:
# create test_dataset object
test_dataset = CustomDataset(X_test, y_test)

In [15]:
# create train and test loader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, pin_memory = True)

In [16]:
class MyNN(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10) # don't need to add softmax layer
        )

    def forward(self, x):
        return self.model(x)


In [17]:
epochs = 100
learning_rate = 0.1

In [18]:
model = MyNN(X_train.shape[1])

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [19]:
for epoch in range(epochs):

    total_epoch_loss = 0
    
    for batch_features, batch_labels in train_loader:

        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)

        loss = criterion(outputs, batch_labels)

        #back pass
        
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step() # updating gradient

        total_epoch_loss += loss.item()

    avg_loss = total_epoch_loss/len(train_loader)

    print(f'Epoch: {epoch+1}, Loss: {avg_loss}')
        

Epoch: 1, Loss: 0.9019714527924856
Epoch: 2, Loss: 0.5293200555642446
Epoch: 3, Loss: 0.4630549712181091
Epoch: 4, Loss: 0.42495631714661913
Epoch: 5, Loss: 0.4022843105395635
Epoch: 6, Loss: 0.38550322751204175
Epoch: 7, Loss: 0.3660860001643499
Epoch: 8, Loss: 0.35455395424366
Epoch: 9, Loss: 0.3428513403336207
Epoch: 10, Loss: 0.33357725163300833
Epoch: 11, Loss: 0.3241902690331141
Epoch: 12, Loss: 0.313672908782959
Epoch: 13, Loss: 0.30772691822052
Epoch: 14, Loss: 0.29807130150000255
Epoch: 15, Loss: 0.2954384444554647
Epoch: 16, Loss: 0.2881200405955315
Epoch: 17, Loss: 0.28084643971920015
Epoch: 18, Loss: 0.2744240506887436
Epoch: 19, Loss: 0.2708665322462718
Epoch: 20, Loss: 0.26529519534111023
Epoch: 21, Loss: 0.2584998475710551
Epoch: 22, Loss: 0.25400416731834413
Epoch: 23, Loss: 0.2490715756813685
Epoch: 24, Loss: 0.24230582696199418
Epoch: 25, Loss: 0.2404193305770556
Epoch: 26, Loss: 0.23457880459229152
Epoch: 27, Loss: 0.23248336839675904
Epoch: 28, Loss: 0.2286103118260

In [20]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [21]:
total = 0
correct = 0

with torch.no_grad():

    for batch_features, batch_labels in test_loader:

        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)

        _ , predicted = torch.max(outputs, 1)

        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

print(correct/total)

0.88925


In [22]:
total = 0
correct = 0

with torch.no_grad():

    for batch_features, batch_labels in train_loader:

        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)

        _ , predicted = torch.max(outputs, 1)

        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

print(correct/total)

0.9775833333333334
